## Email Analysis *Extract, Load, and Transform* Notebook

### Imports

In [1]:
# Standard Library Imports
import logging
from typing import List, Dict, Any
from datetime import datetime
import time

# Third-Party Imports
import pypff # type: ignore
from sqlalchemy.exc import SQLAlchemyError
from tqdm.notebook import tqdm

# Local Imports
from src.utils.config import Config
from src.extract.pst_message_extractor import PstMessageExtractor
from src.transform.message_parser import MessageParser, ParsedMessage
from src.transform.message_enricher import MessageEnricher, EnrichedMessage
from src.load.data_loader import DataLoader
from src.utils.pypff_debug_utils import PST_DEBUG_TOOL

logging.basicConfig(level=logging.INFO)

### Configuration

In [2]:
config: Config = Config.from_json("config.json")
extractor: PstMessageExtractor = PstMessageExtractor(config.input_pst_path, config.chunk_size)
message_parser: MessageParser = MessageParser()
message_enricher: MessageEnricher = MessageEnricher()
loader: DataLoader = DataLoader(host=config.db_host, user=config.db_user, password=config.db_password, database=config.db_name)
# pst_debug: PST_DEBUG_TOOL = PST_DEBUG_TOOL(config.input_pst_path)

### ETL Pipeline with Error Handling

In [3]:
loader.clear_tables()
loader.create_tables()

total_batches = extractor.get_total_batches("Inbox")
pbar = tqdm(total=total_batches, desc='Extracting messages')
start_time = time.time()

logging.info("Starting ETL Pipeline")
for message_batch in extractor.extract_messages("Inbox"):
    enriched_messages: List[EnrichedMessage] = []
    
    for pst_message in message_batch.messages:
        provider_email_id = pst_message.provider_email_id
        # logging.debug(f"Processing message {provider_email_id}")
        try:
            parsed_message: ParsedMessage = message_parser.parse(pst_message.message, provider_email_id, pst_message.folder_name)
        except Exception as e:
            continue

        try:
            enriched_message: EnrichedMessage = message_enricher.enrich_message(parsed_message)
        except Exception as e:
            continue
        else:
            enriched_messages.append(enriched_message)

    try:
        loader.load(enriched_messages)
    except SQLAlchemyError as e:
        logging.error(f"Error loading messages in batch {message_batch.batch_id}: {e}")
        continue
    else:
        logging.debug(f"Loaded {len(enriched_messages)} messages in batch {message_batch.batch_id}")

    pbar.update(1)
end_time = time.time()
pbar.close()
logging.info(f"Finished ETL Pipeline in {int(end_time - start_time) // 60} minutes and {int(end_time - start_time) % 60} seconds")

loader.export_to_excel("./data/processed")
loader.close()

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'localhost' ([Errno 111] Connection refused)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)